In [1]:
import pandas as pd
import numpy as np

In [3]:
#Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
pd.set_option('display.max_columns', None)

In [31]:
df = pd.read_csv("../data/HR_RAW_DATA.csv",index_col=0)
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,...,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",...,20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",...,33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,1,"319,0",...,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,NaN,1
3,47,No,travel_rarely,"359,0$",NaN,2,4,Medical,1,NaN,...,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,NaN,False
4,46,No,NaN,"1319,0$",NaN,3,3,Technical Degree,1,NaN,...,19,NaN,2,8,NaN,1977,1000000000$,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,travel_rarely,"530,0$",NaN,3,1,Life Sciences,1,"967,0",...,16,NaN,3,7,"10325,0",1987,1000000000$,NaN,NaN,0
1610,45,No,non-travel,"805,0$",NaN,4,2,NaN,1,"972,0",...,9,NaN,0,8,"4447,0",1978,1000000000$,NaN,NaN,1
1611,39,No,travel_rarely,"903,0$",NaN,-13,5,NaN,1,NaN,...,8,NaN,0,7,NaN,1984,1000000000$,NaN,NaN,Yes
1612,36,No,non-travel,"1229,0$",NaN,8,4,Technical Degree,1,"990,0",...,7,NaN,0,7,NaN,1987,1000000000$,NaN,NaN,True


In [5]:
df_nulos = pd.DataFrame((df.isnull().sum() / df.shape[0]) * 100, columns = ["%_nulos"])

# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
BusinessTravel,47.831475
Department,81.288724
EducationField,46.158612
employeenumber,26.703841
MaritalStatus,40.334572
MonthlyIncome,52.230483
Over18,55.824040
OverTime,41.883519
PerformanceRating,12.081784
StandardHours,74.039653


In [ ]:
df.drop("NUMBERCHILDREN", axis=1, inplace = True)

In [33]:
indices_dup=df[df.duplicated(subset= 'employeenumber')].index

In [34]:
len(indices_dup)

534

In [9]:
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['BusinessTravel', 'Department', 'EducationField', 'employeenumber',
       'MaritalStatus', 'MonthlyIncome', 'Over18', 'OverTime',
       'PerformanceRating', 'StandardHours', 'TOTALWORKINGYEARS',
       'WORKLIFEBALANCE', 'YearsInCurrentRole', 'SameAsMonthlyIncome',
       'RoleDepartament'],
      dtype='object')


In [17]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'RemoteWork'],
      dtype='object')

In [ ]:
df.duplicated().index

In [13]:
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,...,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",...,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",...,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,1,"319,0",...,NaN,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,1
3,47,No,travel_rarely,"359,0$",NaN,2,4,Medical,1,NaN,...,NaN,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,False
4,46,No,NaN,"1319,0$",NaN,3,3,Technical Degree,1,NaN,...,"3,0",19,NaN,2,8,NaN,1977,1000000000$,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,travel_rarely,"530,0$",NaN,3,1,Life Sciences,1,"967,0",...,"3,0",16,NaN,3,7,"10325,0",1987,1000000000$,NaN,0
1610,45,No,non-travel,"805,0$",NaN,4,2,NaN,1,"972,0",...,"2,0",9,NaN,0,8,"4447,0",1978,1000000000$,NaN,1
1611,39,No,travel_rarely,"903,0$",NaN,-13,5,NaN,1,NaN,...,"3,0",8,NaN,0,7,NaN,1984,1000000000$,NaN,Yes
1612,36,No,non-travel,"1229,0$",NaN,8,4,Technical Degree,1,"990,0",...,"3,0",7,NaN,0,7,NaN,1987,1000000000$,NaN,True


In [ ]:
df.duplicated(subset=df.columns.difference(Index)).sum()

In [18]:
df.drop_duplicates(subset=['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'RemoteWork'])

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,...,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",...,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",...,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,1,"319,0",...,NaN,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,1
3,47,No,travel_rarely,"359,0$",NaN,2,4,Medical,1,NaN,...,NaN,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,False
4,46,No,NaN,"1319,0$",NaN,3,3,Technical Degree,1,NaN,...,"3,0",19,NaN,2,8,NaN,1977,1000000000$,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,travel_rarely,"530,0$",NaN,3,1,Life Sciences,1,"967,0",...,"3,0",16,NaN,3,7,"10325,0",1987,1000000000$,NaN,0
1610,45,No,non-travel,"805,0$",NaN,4,2,NaN,1,"972,0",...,"2,0",9,NaN,0,8,"4447,0",1978,1000000000$,NaN,1
1611,39,No,travel_rarely,"903,0$",NaN,-13,5,NaN,1,NaN,...,"3,0",8,NaN,0,7,NaN,1984,1000000000$,NaN,Yes
1612,36,No,non-travel,"1229,0$",NaN,8,4,Technical Degree,1,"990,0",...,"3,0",7,NaN,0,7,NaN,1987,1000000000$,NaN,True


In [39]:
df[df.duplicated(subset="employeenumber", keep=False)].sort_values(by="employeenumber")



,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
987,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,Not Available,2,1,sALES ReprEsentatiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sALES ReprEsentatiVE - Sales,NaN,False
1483,33,No,non-travel,"1038,0$",Sales,8,1,Life Sciences,1,"1044,0",2,1,88,2,1,sales repResENTaTiVE,4,NaN,NaN,21437,0,Y,NaN,19,"3,0",4,NaN,0,NaN,2,"2,0",2,NaN,2,2,NaN,1990,1000000000$,sales repResENTaTiVE - Sales,NaN,0
1484,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,reSEArcH SCiEnTIst,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,Yes
988,26,Yes,NaN,"342,0$",NaN,2,3,Life Sciences,1,"1053,0",1,0,57,3,1,rEsEaRCH SCIenTiSt,1,Married,"2042,0",15346,6,Y,NaN,14,"3,0",2,NaN,1,"6,0",2,"3,0",3,NaN,1,2,"2042,0",1997,1000000000$,NaN,NaN,1
990,46,No,NaN,"430,0$",NaN,-47,4,Medical,1,"1069,0",47,0,40,3,5,RESeArCH dIrEcTOR,4,NaN,NaN,21445,9,NaN,No,17,"3,0",4,"80,0",2,"23,0",0,"3,0",2,NaN,2,2,NaN,1977,1000000000$,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,28,No,NaN,"760,0$",Sales,-43,4,NaN,1,NaN,43,1,81,3,2,SaLEs ExECUtiVE,2,NaN,NaN,3698,1,Y,NaN,20,"4,0",1,NaN,0,"8,0",2,"3,0",8,NaN,7,5,NaN,1995,1000000000$,SaLEs ExECUtiVE - Sales,NaN,0
1602,26,Yes,NaN,nan$,NaN,5,2,Medical,1,NaN,3,1,88,2,1,reSearCH scientiST,3,Married,"2366,0",20898,1,NaN,NaN,14,"3,0",1,NaN,1,"8,0",2,"3,0",8,NaN,1,7,"2366,0",1997,1000000000$,NaN,NaN,True
1607,31,No,NaN,"616,0$",NaN,12,3,Medical,1,NaN,4,1,41,3,2,HEaltHCarE REPreSENtAtIve,4,NaN,NaN,17369,0,NaN,NaN,11,"3,0",3,NaN,2,"10,0",2,"1,0",9,NaN,8,5,NaN,1992,1000000000$,NaN,NaN,True
1611,39,No,travel_rarely,"903,0$",NaN,-13,5,NaN,1,NaN,13,0,41,4,3,sAlES ExECUTivE,3,Single,NaN,2560,0,NaN,No,18,"3,0",4,NaN,0,"9,0",3,"3,0",8,NaN,0,7,NaN,1984,1000000000$,NaN,NaN,Yes


In [ ]:
df.duplicated(subset= )

In [40]:
df["HourlyRate"].value_counts()

Not Available    84
42               33
66               32
48               30
57               29
                 ..
47               15
53               13
68               13
38               12
34               11
Name: HourlyRate, Length: 72, dtype: int64